<a href="https://colab.research.google.com/github/buddypia/openai-agents-sdk-colab/blob/master/llm_as_a_judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLM-as-a-Judge

作成された物語を評価し、80点以下の場合はフィードバックとともに改善を行い、80点を超えた場合に最終的な物語を表示する


## INPUT
```markdown
桃太郎の物語
```

## OUTPUT
```markdown
80点超過の物語を表示
```

## エージェントフロー図
<img src="https://drive.google.com/uc?id=1fKLrRYQSSGbYLahw6Ry_Ky_1zw5zGem3" width="80%">

In [ ]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

# ColabのシークレットからOpenAIのAPIキーを取得
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# OpenAIエージェントにAPIキーを設定
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
from agents import Agent

"""
この例は、LLMを判定者として使用するパターンを示しています。
最初のエージェントが物語の概要を生成します。
2番目のエージェントがその概要を評価し、フィードバックを提供します。
判定者が概要に満足するまでループします。
"""

story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "ユーザーの入力に基づいて非常に短い物語の概要を生成します。"
        "フィードバックが提供された場合は、それを使用して概要を改善します。"
    ),
)

In [ ]:
from dataclasses import dataclass
from typing import Literal

@dataclass
class EvaluationFeedback:
    feedback: str
    score: int


evaluator = Agent[None](
    name="evaluator",
    instructions=(
        "物語の概要を評価し、それが十分な品質かどうかを判断します。"
        "1点から100点で評価します。"
        "評価が十分でない場合は、改善が必要な点についてフィードバックを提供します。"
        "最初の試行で100点をさせることは絶対にありません。"
    ),
    output_type=EvaluationFeedback,
)

In [ ]:
query = "桃太郎の物語"

In [ ]:
from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

input_items: list[TResponseInputItem] = [{"content": query, "role": "user"}]

for attempt in range(10):
    story_outline_result = await Runner.run(
        story_outline_generator,
        input_items,
    )
    print(f"{attempt + 1}回目のアウトライン: {story_outline_result.final_output}")

    input_items = story_outline_result.to_input_list()
    latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)

    evaluator_result = await Runner.run(evaluator, input_items)
    result: EvaluationFeedback = evaluator_result.final_output

    print(f"評価者のスコア: {result.score}、フィードバック: {result.feedback}")

    if result.score > 80:
        print("物語の概要は十分です。終了します。")
        print("----------------------------------")
        break

    print("フィードバックを反映して再実行します")
    print("----------------------------------")

    input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})

print(f"最終的な物語の概要: {latest_outline}")


1回目のアウトライン: ある日、桃が川を流れてきて、おばあさんが見つけます。その桃を割ると、中から男の子、桃太郎が現れます。成長した桃太郎は、鬼退治に旅立ちます。道中で犬、猿、キジを仲間に加え、共に鬼ヶ島の鬼を退治し、村に平和を取り戻します。
評価者のスコア: 75、フィードバック: この物語は、桃太郎のあらすじを簡潔にまとめています。

改善点：
1. 背景情報：おじいさんやおばあさんの境遇や心情が不明なので、もう少し詳細を加えると物語に厚みが出ます。
2. キャラクターの動機：桃太郎がなぜ鬼退治に行くのか、その動機を詳しく描くと物語に説得力が増します。
3. クライマックスの描写：鬼との戦いの場面や戦いの過程をもう少し詳しく描くと、物語に緊張感と魅力が増すでしょう。

これらを考慮して調整すると、より豊かで引き込まれる物語になるでしょう。
フィードバックを反映して再実行します
----------------------------------
2回目のアウトライン: 昔々、心優しいおじいさんとおばあさんが山奥で慎ましく暮らしていました。二人は子供が欲しいと願い続けていました。ある日、川で洗濯をしていたおばあさんは巨大な桃を見つけ、家に持ち帰ります。桃を割ると、中から元気な男の子が現れ、桃太郎と名付けられました。

成長した桃太郎は、村を襲う鬼の噂を耳にし、村の安全を守るために鬼退治を決意します。犬、猿、キジが協力を申し出て、桃太郎の仲間になります。

鬼ヶ島に到着した一行は、知恵と力を合わせて鬼たちと戦います。激しい戦いの末、桃太郎たちは見事に鬼を打ち倒し、村から奪われた宝物を取り戻しました。村に平和を取り戻した桃太郎たちは、みんなから感謝され、その後も仲間と共に幸せに暮らしました。
評価者のスコア: 85、フィードバック: この物語は、桃太郎のあらすじを簡潔にまとめています。

改善点：
1. 背景情報：おじいさんやおばあさんの境遇や心情が不明なので、もう少し詳細を加えると物語に厚みが出ます。
2. キャラクターの動機：桃太郎がなぜ鬼退治に行くのか、その動機を詳しく描くと物語に説得力が増します。
3. クライマックスの描写：鬼との戦いの場面や戦いの過程をもう少し詳しく描くと、物語に緊張感と魅力が増すでしょう。

これらを考慮して調整すると、より豊かで引き込ま